In [4]:
from src.knn_methods import KNNClassifier
from src.dataset import SwissRoll
import pandas as pd
import graphtools as gt
import os
import pickle
from omegaconf import DictConfig, OmegaConf
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from src.sc_dataset import EBData, AdataTraj, MultiCiteSeq
import numpy as np
from src.embedding import new_HeatGeo
from src.emd import earth_movers_distance,sinkhorn_mccann
import ot
from sklearn import metrics
import numpy as np
from omegaconf import OmegaConf



load_dotenv()
DATA_DIR = os.environ["SHARE_DATA"]

In [7]:
! ls $DATA_DIR

 adata			       MNIST
 adata_ec1_v1.h5ad	       models
 cfm_last.ckpt		       multiome_gex_processed_training.h5ad
 cifar-10-batches-py	       op_cite_inputs_0.h5ad
 cifar-10-python.tar.gz        op_cite_inputs_1.h5ad
 cifar10_stats.npz	       op_cite_inputs_2.h5ad
 clark.npz		       op_cite_inputs_3.h5ad
 eb_velocity_v5.npz	       op_cite_inputs.h5ad
 fid_stats_cifar10_train.npz  'open?id=14UB27-Spi8VjZYKST3ZcT8YVhAluiFWI'
 gaussians_20dim_back.pkl      op_train_multi_targets_0.h5ad
 gaussians_2dim_back.pkl       op_train_multi_targets_1.h5ad
 gaussians_50dim_back.pkl      op_train_multi_targets_2.h5ad
 gaussians_5dim_back.pkl       op_train_multi_targets.h5ad
 gh_last.ckpt		       slurm-2958990.out
 last.ckpt		       wot_v1.h5ad


RESULTS

In [9]:
# load results and check in config files the knn, tau, and emb_dim.
def load_results(path):
    results = pd.DataFrame()
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith("_0.pkl"):
                last_is_config = False
                dt_current = pd.read_pickle(os.path.join(root, filename))
                print(os.path.join(root, filename), "res")
            if filename.endswith("config.yaml") and not last_is_config:
                last_is_config = True
                config = OmegaConf.load(os.path.join(root, filename))
                print(os.path.join(root, filename), "config")
                dataset_name = config.dataset_name
                config = config.model
                order = 0 if "order" not in config else config.order
                order = 0 if order is None else order
                tau = 0 if "tau" not in config.keys() else config.tau
                knn = 0 if "knn" not in config.keys() else config.knn
                n_neighbors = 0 if "n_neighbors" not in config.keys() else config.n_neighbors
                emb_dim = 0 if "emb_dim" not in config.keys() else config.emb_dim
                n_components = 0 if "n_components" not in config.keys() else config.n_components
                perplexity = 0 if "perplexity" not in config.keys() else config.perplexity
                min_dist = 0 if "min_dist" not in config.keys() else config.min_dist
                denoising = 0 if "denoising" not in config.keys() else config.denoising

                # print(order, tau, knn, emb_dim, n_neighbors, n_components)
                dt_current["order"] = order
                dt_current["tau"] = tau
                dt_current["knn"] = knn
                dt_current["emb_dim"] = emb_dim
                dt_current["n_neighbors"] = n_neighbors
                dt_current["n_components"] = n_components
                dt_current["dataset_name"] = dataset_name
                dt_current["perplexity"] = perplexity
                dt_current["min_dist"] = min_dist
                dt_current["denoising"] = denoising
                results = pd.concat([results,dt_current])
    return results

In [11]:
res = load_results("/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/runs/interpolation")
res 

/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/runs/interpolation/2023-05-09_18-49-48/ot_pred_EB_heat_geo_euler_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/runs/interpolation/2023-05-09_18-49-48/.hydra/config.yaml config


,Method,Seed,Holdout,W1,W1,MMD RBF,MMD Linear,order,tau,knn,emb_dim,n_neighbors,n_components,dataset_name,perplexity,min_dist,denoising
0,heat_geo_euler,0,1,1.677962,3.176551,0.221050,1.516787,30,0.05,10,5,0,0,EB,0,0,False
1,heat_geo_euler,0,2,1.467934,2.362156,0.105140,0.531322,30,0.05,10,5,0,0,EB,0,0,False
2,heat_geo_euler,0,3,1.432773,2.258759,0.091692,0.450973,30,0.05,10,5,0,0,EB,0,0,False


TESTING THE CODE

In [45]:
! python /network/scratch/g/guillaume.huguet/projects/metric_embeddings/ot_task.py

2023-05-09 15:04:11.624278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 15:04:15.612174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-09 15:04:15.612811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-09 15:04:15.612832: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))

    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]

    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})

    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()

def mmd_linear(X, Y):
    """MMD using linear kernel (i.e., k(x,y) = <x,y>)
    Note that this is not the original linear MMD, only the reformulated and faster version.
    The original version is:
        def mmd_linear(X, Y):
            XX = np.dot(X, X.T)
            YY = np.dot(Y, Y.T)
            XY = np.dot(X, Y.T)
            return XX.mean() + YY.mean() - 2 * XY.mean()

    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]

    Returns:
        [scalar] -- [MMD value]
    """
    delta = X.mean(0) - Y.mean(0)
    return delta.dot(delta.T)

In [23]:
DATA_DIR

'/network/scratch/a/alexander.tong/trajectory-inference/data'

In [33]:
# TESTING the interpolation method
#n_points = 1e10
n_points = 2000
#ds = EBData(n_points=n_points, random_state=42)
#ds = MultiCiteSeq(data_file = "op_train_multi_targets_0.h5ad",n_points=n_points, random_state=42)
ds = MultiCiteSeq(data_file = "wot_v1.h5ad",n_points=n_points, random_state=42)
data, labels = ds.X, ds.labels

In [34]:
data.shape

(2000, 100)

In [35]:
unique_time = np.unique(labels)
hold_out = unique_time[1:-1]

In [36]:
unique_time


array([ 0.  ,  0.5 ,  1.  ,  1.5 ,  2.  ,  2.5 ,  3.  ,  3.5 ,  4.  ,
        4.5 ,  5.  ,  5.5 ,  6.  ,  6.5 ,  7.  ,  7.5 ,  8.  ,  8.25,
        8.5 ,  8.75,  9.  ,  9.5 , 10.  , 10.5 , 11.  , 11.5 , 12.  ,
       12.5 , 13.  , 13.5 , 14.  , 14.5 , 15.  , 15.5 , 16.  , 16.5 ,
       17.  , 17.5 , 18.  ])

In [37]:
emb = new_HeatGeo(knn=5, emb_dim=2).fit_transform(data)

Using s_gd2 for MDS. None


In [38]:
emb.shape

(2000, 2)

In [39]:
def eval_interpolation(pred,true, n_dim=5):
    dist_s = ot.dist(pred, true, metric='sqeuclidean')
    dist = ot.dist(pred, true, metric='euclidean')

    a = np.ones(pred.shape[0])/pred.shape[0]
    b = np.ones(true.shape[0])/true.shape[0]
    w2 = ot.emd2(a, b, dist_s)
    w1 = ot.emd2(a, b, dist)


    mmd_r = mmd_rbf(pred, true, gamma=1/n_dim)

    mmd_l = mmd_linear(pred, true)

    return [w1, w2, mmd_r, mmd_l]


    

In [40]:
col = ["w1", "w2", "mmd_r", "mmd_l"]
df = pd.DataFrame(columns=col)

norm_emb = (emb - emb.mean(0))/emb.std(0)

for idx_ho in range(1,len(unique_time[:-1])):
    t_init = unique_time[idx_ho-1]
    t_hold = unique_time[idx_ho]
    t_final = unique_time[idx_ho+1]

    mask_init = labels==t_init
    mask_hold = labels==t_hold
    mask_final = labels==t_final

    emb_init = norm_emb[mask_init]
    emb_hold = norm_emb[mask_hold]
    emb_final = norm_emb[mask_final]

    pred_ho = sinkhorn_mccann(emb_init, emb_hold, emb_final, n_points=5000)
    res = eval_interpolation(pred_ho, emb_hold)
    df_run = pd.DataFrame([res], columns=col)
    df = pd.concat([df,df_run], ignore_index=True)
    